# Trading Strategy Analysis Framework

## Overview
This project implements and analyzes four different trading strategies based on MACD (Moving Average Convergence Divergence) and VPVMA (Volume-Price-Volatility Moving Average) indicators. The strategies are tested on weekly market data with VIX integration for volatility awareness.

## Strategies Implemented

### 1. Traditional MACD
- Uses standard MACD crossover signals
- 12/26/9 period settings for EMA calculations
- Weekly rebalancing with 1-week signal lag
- 5% stop-loss protection

### 2. MACD Zero-Cross
- Enhanced MACD strategy requiring:
  - MACD line crosses above/below signal line
  - MACD must be above zero for long positions
  - MACD must be below zero for short positions
- More conservative approach to reduce false signals

### 3. VPVMA (Volume-Price-Volatility Moving Average)
- Novel approach combining:
  - Volume-weighted typical price
  - VIX-based volatility adjustment
  - Similar signal generation to MACD
- Aims to capture both price momentum and market sentiment

### 4. VPVMA Zero-Cross
- Enhanced VPVMA strategy with zero-line confirmation
- Requires both signal line crossover and zero-line validation
- More selective entry/exit points

## Technical Implementation

The framework includes:
- Parallel processing of multiple strategies
- Automated data downloading using yfinance
- Timezone-aware calculations
- Comprehensive performance metrics
- Stop-loss implementation using intraweek price data

## Performance Analysis

For each strategy, the framework calculates:
- Sharpe Ratio (annualized)
- Portfolio returns
- Strategy comparison metrics
- Best performing strategy identification

Results are automatically saved to:
- CSV files for detailed analysis
- Strategy comparison text files
- Organized directory structure by symbol


## Risk Management

All strategies incorporate:
- 5% stop-loss protection
- Position sizing based on portfolio value
- One-week signal lag for realistic implementation
- Intraweek price monitoring for stop-loss triggers

## Data Requirements
- Historical price data (OHLCV)
- VIX data for volatility calculations
- Minimum 5 years of data recommended for reliable backtesting

## Future Improvements
- [ ] Add position sizing optimization
- [ ] Implement dynamic stop-loss based on volatility
- [ ] Add more sophisticated portfolio management rules
- [ ] Include transaction costs and slippage
- [ ] Add cross-validation periods

## Directory Structure
data/\
├── {SYMBOL}/\
│   ├── weekly_macd_signals.csv\
│   ├── weekly_macd_zero_cross.csv\
│   ├── weekly_vpvma_signals.csv\
│   ├── weekly_vpvma_zero_cross.csv\
│   └── strategy_comparison.txt\



In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import os
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

/home/vahid/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
def download_data(symbol, start_date, end_date):
    """Download price and VIX data for a symbol"""
    ticker = yf.Ticker(symbol)
    df = ticker.history(start=start_date, end=end_date)
    
    # Download VIX data only once if needed
    if symbol not in ['VIX', '^VIX']:
        vix = yf.Ticker('^VIX')
        vix_df = vix.history(start=start_date, end=end_date)
        return df, vix_df
    return df, None

In [3]:
def analyze_strategy_performance(results, symbol):
    """Analyze and compare strategy performance for a ticker"""
    strategy_metrics = {
        'MACD': results[0],
        'MACD Zero-Cross': results[1],
        'VPVMA': results[2],
        'VPVMA Zero-Cross': results[3]
    }
    
    # Calculate Sharpe ratio for each strategy
    sharpe_ratios = {}
    for strategy_name, df in strategy_metrics.items():
        returns = df['Strategy_Returns']
        sharpe = np.sqrt(52) * returns.mean() / returns.std() if returns.std() != 0 else 0
        sharpe_ratios[strategy_name] = sharpe
    
    # Find best strategy
    best_strategy = max(sharpe_ratios.items(), key=lambda x: x[1])
    
    # Save performance comparison to ticker directory
    ticker_dir = os.path.join('data', symbol.replace('^', ''))
    performance_file = os.path.join(ticker_dir, 'strategy_comparison.txt')
    
    with open(performance_file, 'w') as f:
        f.write(f"Strategy Performance Comparison for {symbol}\n")
        f.write("=" * 50 + "\n\n")
        f.write("Sharpe Ratios:\n")
        for strategy, sharpe in sharpe_ratios.items():
            f.write(f"{strategy}: {sharpe:.2f}\n")
        f.write(f"\nBest Strategy: {best_strategy[0]} (Sharpe: {best_strategy[1]:.2f})")
    
    return best_strategy[0], sharpe_ratios

In [4]:
def process_etf(symbol, start_date='2005-01-01', end_date='2023-12-31', initial_capital=1_000_000):
    """Process all strategies for a single ETF"""
    try:
        # Create ETF-specific directory
        ticker_dir = os.path.join('data', symbol.replace('^', ''))
        os.makedirs(ticker_dir, exist_ok=True)
        
        # Download data once and reuse
        df, vix_df = download_data(symbol, start_date, end_date)
        
        results = []
        results.append(get_macd_signals(df=df.copy(), symbol=symbol))
        results.append(get_macd_signals_zero_cross(df=df.copy(), symbol=symbol))
        results.append(get_vpvma_signals(df=df.copy(), vix_df=vix_df.copy(), symbol=symbol))
        results.append(get_vpvma_signals_zero_cross(df=df.copy(), vix_df=vix_df.copy(), symbol=symbol))
        
        # Analyze strategy performance
        best_strategy, sharpe_ratios = analyze_strategy_performance(results, symbol)
        print(f"\n{symbol} Best Strategy: {best_strategy}")
        
        return results, best_strategy, sharpe_ratios
            
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        return None

In [5]:
def apply_stop_loss(df, stop_loss_pct=0.03):
    """
    Apply stop loss to positions immediately when threshold is breached
    Uses intraweek high/low prices to check for stop loss triggers
    Returns a new DataFrame with stop loss applied
    """
    # Create a copy of the input DataFrame
    result_df = df.copy()
    
    position = 0
    entry_price = 0
    portfolio_value = result_df['Portfolio_Value'].iloc[0]
    
    for i in range(len(result_df)):
        if result_df['Position'].iloc[i] != 0 and position == 0:
            # Enter new position
            position = result_df['Position'].iloc[i]
            entry_price = result_df['Close'].iloc[i]
            portfolio_value = result_df['Portfolio_Value'].iloc[i]
        elif position != 0:
            # Check for stop loss using High and Low prices
            if position == 1:  # Long position
                lowest_price = result_df['Low'].iloc[i]
                loss_pct = (lowest_price - entry_price) / entry_price
                if loss_pct < -stop_loss_pct:
                    # Stop loss triggered - use the stop loss price for return calculation
                    stop_price = entry_price * (1 - stop_loss_pct)
                    result_df.loc[result_df.index[i], 'Close'] = stop_price  # Assume execution at stop price
                    result_df.loc[result_df.index[i], 'Position'] = 0
                    position = 0
                    entry_price = 0
                    
            else:  # Short position
                highest_price = result_df['High'].iloc[i]
                loss_pct = (entry_price - highest_price) / entry_price
                if loss_pct < -stop_loss_pct:
                    # Stop loss triggered - use the stop loss price for return calculation
                    stop_price = entry_price * (1 + stop_loss_pct)
                    result_df.loc[result_df.index[i], 'Close'] = stop_price  # Assume execution at stop price
                    result_df.loc[result_df.index[i], 'Position'] = 0
                    position = 0
                    entry_price = 0
            
            # Check for regular position change
            if result_df['Position'].iloc[i] != position and position != 0:
                position = result_df['Position'].iloc[i]
                entry_price = result_df['Close'].iloc[i] if position != 0 else 0
                portfolio_value = result_df['Portfolio_Value'].iloc[i]
    
    return result_df

In [6]:
def calculate_strategy_returns(df):
    """Calculate strategy returns with position changes"""
    df['Returns'] = df['Close'].pct_change()
    df['Strategy_Returns'] = df['Position'] * df['Returns']
    df['Strategy_Returns'] = df['Strategy_Returns'].fillna(0)
    df['Portfolio_Returns'] = df['Strategy_Returns']
    df['Portfolio_Value'] = df['Portfolio_Value'].iloc[0] * (1 + df['Portfolio_Returns']).cumprod()
    df['Position_Change'] = df['Position'].diff()
    return df

In [7]:
def get_macd_signals(df=None, symbol='^GSPC', start_date='2005-01-01', end_date='2023-12-31', initial_capital=1_000_000):
    """MACD strategy with pre-downloaded data option"""
    if df is None:
        ticker = yf.Ticker(symbol)
        df = ticker.history(start=start_date, end=end_date)
    
    # Convert timezone from UTC to US/Eastern
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_convert('US/Eastern')
    
    # Resample to weekly data (last trading day of the week)
    weekly_df = df.resample('W').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })
    
    # Calculate weekly MACD
    exp1 = weekly_df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = weekly_df['Close'].ewm(span=26, adjust=False).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span=9, adjust=False).mean()
    
    # Create signals on weekly data
    weekly_df['MACD'] = macd
    weekly_df['Signal_Line'] = signal
    weekly_df['MACD_Histogram'] = macd - signal
    
    # Generate buy/sell signals
    weekly_df['Position'] = 0
    weekly_df['Position'] = weekly_df['Position'].mask(macd > signal, 1)
    weekly_df['Position'] = weekly_df['Position'].mask(macd < signal, -1)
    
    # Shift positions by 1 week to implement signal lag
    weekly_df['Position'] = weekly_df['Position'].shift(1)
    
    # Initialize Portfolio Value
    weekly_df['Portfolio_Value'] = 1_000_000
    
    # Apply stop loss with 5%
    weekly_df = apply_stop_loss(weekly_df, stop_loss_pct=0.05)
    
    # Recalculate returns after stop loss
    weekly_df = calculate_strategy_returns(weekly_df)
    
    # Save DataFrame to CSV in ticker directory
    output_file = os.path.join('data', symbol.replace('^', ''), 'weekly_macd_signals.csv')
    weekly_df.to_csv(output_file)
    print(f"Data saved to {output_file}")
    
    return weekly_df

In [8]:
def get_macd_signals_zero_cross(df, symbol):
    # Convert timezone from UTC to US/Eastern
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_convert('US/Eastern')
    
    # Resample to weekly data (last trading day of the week)
    weekly_df = df.resample('W').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })
    
    # Calculate weekly MACD
    exp1 = weekly_df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = weekly_df['Close'].ewm(span=26, adjust=False).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span=9, adjust=False).mean()
    
    # Create signals on weekly data
    weekly_df['MACD'] = macd
    weekly_df['Signal_Line'] = signal
    weekly_df['MACD_Histogram'] = macd - signal
    
    # Generate buy/sell signals with zero-line condition
    weekly_df['Position'] = 0
    
    # Previous position to maintain when no new signal
    prev_position = 0
    
    for i in range(len(weekly_df)):
        # Buy signal: MACD crosses above signal line AND MACD is above zero
        if (macd.iloc[i] > signal.iloc[i]) and (macd.iloc[i] > 0):
            weekly_df.iloc[i, weekly_df.columns.get_loc('Position')] = 1
            prev_position = 1
        # Sell signal: MACD crosses below signal line AND MACD crosses below zero
        elif (macd.iloc[i] < signal.iloc[i]) and (macd.iloc[i] < 0):
            weekly_df.iloc[i, weekly_df.columns.get_loc('Position')] = -1
            prev_position = -1
        else:
            # Maintain previous position when no new signal
            weekly_df.iloc[i, weekly_df.columns.get_loc('Position')] = prev_position
    
    # Shift positions by 1 week to implement signal lag
    weekly_df['Position'] = weekly_df['Position'].shift(1)
    
    # Initialize Portfolio Value
    weekly_df['Portfolio_Value'] = 1_000_000
    
    # Apply stop loss with 5%
    weekly_df = apply_stop_loss(weekly_df, stop_loss_pct=0.05)
    
    # Recalculate returns after stop loss
    weekly_df = calculate_strategy_returns(weekly_df)
    
    # Save DataFrame to CSV in ticker directory
    output_file = os.path.join('data', symbol.replace('^', ''), 'weekly_macd_zero_cross.csv')
    weekly_df.to_csv(output_file)
    print(f"Data saved to {output_file}")
    
    return weekly_df

In [9]:
def get_vpvma_signals(df=None, vix_df=None, symbol='^GSPC', start_date='2005-01-01', end_date='2023-12-31', initial_capital=1_000_000):
    """VPVMA strategy with pre-downloaded data option"""
    if df is None or vix_df is None:
        ticker = yf.Ticker(symbol)
        df = ticker.history(start=start_date, end=end_date)
        vix = yf.Ticker('^VIX')
        vix_df = vix.history(start=start_date, end=end_date)
    
    # Convert timezone from UTC to US/Eastern
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_convert('US/Eastern')
    vix_df.index = pd.to_datetime(vix_df.index)
    vix_df.index = vix_df.index.tz_convert('US/Eastern')
    
    # Resample to weekly data
    weekly_df = df.resample('W').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })
    
    # Resample VIX to weekly data (using average VIX for the week)
    weekly_vix = vix_df.resample('W').agg({
        'Close': 'mean'  # Using mean of VIX values for the week
    })
    
    # Calculate typical price
    weekly_df['Typical_Price'] = (weekly_df['High'] + weekly_df['Low'] + weekly_df['Close']) / 3
    
    # Use VIX as volatility proxy (divide by 100 to convert percentage to decimal)
    weekly_df['Volatility'] = weekly_vix['Close'] / 100
    
    # Calculate volume-weighted typical price
    weekly_df['Vol_Weighted_Price'] = weekly_df['Typical_Price'] * weekly_df['Volume']
    
    # Calculate short-term (12-week) and long-term (26-week) VPVMA
    short_window = 12
    long_window = 26
    signal_window = 9
    
    # Calculate volume-weighted moving averages
    vwp_short = weekly_df['Vol_Weighted_Price'].rolling(window=short_window).sum() / \
                weekly_df['Volume'].rolling(window=short_window).sum()
    vwp_long = weekly_df['Vol_Weighted_Price'].rolling(window=long_window).sum() / \
               weekly_df['Volume'].rolling(window=long_window).sum()
    
    # Multiply by volatility and apply EMA smoothing
    vpvma = (vwp_short * weekly_df['Volatility']).ewm(span=short_window, adjust=False).mean()
    vpvma_long = (vwp_long * weekly_df['Volatility']).ewm(span=long_window, adjust=False).mean()
    
    # Calculate VPVMA (similar to MACD)
    weekly_df['VPVMA'] = vpvma - vpvma_long
    weekly_df['VPVMA_Signal'] = weekly_df['VPVMA'].ewm(span=signal_window, adjust=False).mean()
    weekly_df['VPVMA_Histogram'] = weekly_df['VPVMA'] - weekly_df['VPVMA_Signal']
    
    # Generate buy/sell signals
    weekly_df['Position'] = 0
    weekly_df['Position'] = weekly_df['Position'].mask(
        (weekly_df['VPVMA'] > weekly_df['VPVMA_Signal']), 1)
    weekly_df['Position'] = weekly_df['Position'].mask(
        (weekly_df['VPVMA'] < weekly_df['VPVMA_Signal']), -1)
    
    # Shift positions by 1 week to implement signal lag
    weekly_df['Position'] = weekly_df['Position'].shift(1)
    
    # Initialize Portfolio Value
    weekly_df['Portfolio_Value'] = 1_000_000
    
    # Apply stop loss with 5%
    weekly_df = apply_stop_loss(weekly_df, stop_loss_pct=0.05)
    
    # Recalculate returns after stop loss
    weekly_df = calculate_strategy_returns(weekly_df)
    
    # Save DataFrame to CSV in ticker directory
    output_file = os.path.join('data', symbol.replace('^', ''), 'weekly_vpvma_signals.csv')
    weekly_df.to_csv(output_file)
    print(f"Data saved to {output_file}")
    
    return weekly_df

In [10]:
def get_vpvma_signals_zero_cross(df, vix_df, symbol):
    # Convert timezone from UTC to US/Eastern
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_convert('US/Eastern')
    vix_df.index = pd.to_datetime(vix_df.index)
    vix_df.index = vix_df.index.tz_convert('US/Eastern')
    
    # Resample to weekly data
    weekly_df = df.resample('W').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })
    
    # Resample VIX to weekly data
    weekly_vix = vix_df.resample('W').agg({
        'Close': 'mean'
    })
    
    # Calculate typical price
    weekly_df['Typical_Price'] = (weekly_df['High'] + weekly_df['Low'] + weekly_df['Close']) / 3
    
    # Use VIX as volatility proxy
    weekly_df['Volatility'] = weekly_vix['Close'] / 100
    
    # Calculate volume-weighted typical price
    weekly_df['Vol_Weighted_Price'] = weekly_df['Typical_Price'] * weekly_df['Volume']
    
    # Calculate short-term and long-term VPVMA
    short_window = 12
    long_window = 26
    signal_window = 9
    
    # Calculate volume-weighted moving averages
    vwp_short = weekly_df['Vol_Weighted_Price'].rolling(window=short_window).sum() / \
                weekly_df['Volume'].rolling(window=short_window).sum()
    vwp_long = weekly_df['Vol_Weighted_Price'].rolling(window=long_window).sum() / \
               weekly_df['Volume'].rolling(window=long_window).sum()
    
    # Multiply by volatility and apply EMA smoothing
    vpvma = (vwp_short * weekly_df['Volatility']).ewm(span=short_window, adjust=False).mean()
    vpvma_long = (vwp_long * weekly_df['Volatility']).ewm(span=long_window, adjust=False).mean()
    
    # Calculate VPVMA
    weekly_df['VPVMA'] = vpvma - vpvma_long
    weekly_df['VPVMA_Signal'] = weekly_df['VPVMA'].ewm(span=signal_window, adjust=False).mean()
    weekly_df['VPVMA_Histogram'] = weekly_df['VPVMA'] - weekly_df['VPVMA_Signal']
    
    # Generate buy/sell signals with zero-line condition
    weekly_df['Position'] = 0
    
    # Previous position to maintain when no new signal
    prev_position = 0
    
    for i in range(len(weekly_df)):
        # Buy signal: VPVMA crosses above signal line AND VPVMA is above zero
        if (weekly_df['VPVMA'].iloc[i] > weekly_df['VPVMA_Signal'].iloc[i]) and (weekly_df['VPVMA'].iloc[i] > 0):
            weekly_df.iloc[i, weekly_df.columns.get_loc('Position')] = 1
            prev_position = 1
        # Sell signal: VPVMA crosses below signal line AND VPVMA crosses below zero
        elif (weekly_df['VPVMA'].iloc[i] < weekly_df['VPVMA_Signal'].iloc[i]) and (weekly_df['VPVMA'].iloc[i] < 0):
            weekly_df.iloc[i, weekly_df.columns.get_loc('Position')] = -1
            prev_position = -1
        else:
            # Maintain previous position when no new signal
            weekly_df.iloc[i, weekly_df.columns.get_loc('Position')] = prev_position
    
    # Shift positions by 1 week to implement signal lag
    weekly_df['Position'] = weekly_df['Position'].shift(1)
    
    # Initialize Portfolio Value
    weekly_df['Portfolio_Value'] = 1_000_000
    
    # Apply stop loss with 5%
    weekly_df = apply_stop_loss(weekly_df, stop_loss_pct=0.05)
    
    # Recalculate returns after stop loss
    weekly_df = calculate_strategy_returns(weekly_df)
    
    # Save DataFrame to CSV in ticker directory
    output_file = os.path.join('data', symbol.replace('^', ''), 'weekly_vpvma_zero_cross.csv')
    weekly_df.to_csv(output_file)
    print(f"Data saved to {output_file}")
    
    return weekly_df

In [11]:
def calculate_performance_metrics(df):
    """Calculate various trading performance metrics"""
    
    # Number of Trades
    trades = df['Position_Change'].fillna(0)
    num_trades = len(trades[trades != 0])
    
    # Win Ratio - only count returns when position changes
    position_changes = df[df['Position_Change'] != 0]
    winning_trades = len(position_changes[position_changes['Strategy_Returns'] > 0])
    win_ratio = winning_trades / num_trades if num_trades > 0 else 0
    
    # Profit & Loss
    cumulative_returns = (1 + df['Strategy_Returns']).cumprod()
    total_return = (cumulative_returns.iloc[-1] - 1) * 100
    
    # Calculate annual return
    years = (df.index[-1] - df.index[0]).days / 365.25
    annual_return = (cumulative_returns.iloc[-1] ** (1/years)) - 1
    
    # Sharpe Ratio (assuming 0% risk-free rate)
    mean_returns = df['Strategy_Returns'].mean()
    std_returns = df['Strategy_Returns'].std()
    sharpe_ratio = np.sqrt(52) * mean_returns / std_returns if std_returns != 0 else 0
    
    # Maximum Drawdown calculation
    portfolio_value = df['Portfolio_Value']
    peak = portfolio_value.expanding(min_periods=1).max()
    drawdown = (portfolio_value - peak) / peak
    max_drawdown = drawdown.min() * 100
    
    # Portfolio Value metrics
    initial_value = df['Portfolio_Value'].iloc[0]
    final_value = df['Portfolio_Value'].iloc[-1]
    portfolio_return = ((final_value - initial_value) / initial_value) * 100
    
    return {
        'Number of Trades': num_trades,
        'Win Ratio': f"{win_ratio:.2%}",
        'Total Return': f"{total_return:.2f}%",
        'Annual Return': f"{annual_return*100:.2f}%",
        'Sharpe Ratio': f"{sharpe_ratio:.2f}",
        'Maximum Drawdown': f"{max_drawdown:.2f}%",
        'Initial Portfolio Value': f"${initial_value:,.2f}",
        'Final Portfolio Value': f"${final_value:,.2f}",
        'Portfolio Return': f"{portfolio_return:.2f}%"
    }

In [12]:
def plot_macd_signals(df):
    """Plot MACD signals and price movements"""
    # Create figure with secondary y-axis
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), height_ratios=[2, 1])
    
    # Plot price
    ax1.plot(df.index, df['Close'], label='Price', color='blue', alpha=0.6)
    
    # Plot buy/sell signals
    buy_signals = df[df['Position_Change'] == 1].index
    sell_signals = df[df['Position_Change'] == -2].index  # From 1 to -1
    ax1.scatter(buy_signals, df.loc[buy_signals, 'Close'], marker='^', color='green', label='Buy Signal')
    ax1.scatter(sell_signals, df.loc[sell_signals, 'Close'], marker='v', color='red', label='Sell Signal')
    
    ax1.set_title('Price Movement and Trading Signals')
    ax1.set_ylabel('Price')
    ax1.legend()
    
    # Plot MACD
    ax2.plot(df.index, df['MACD'], label='MACD', color='blue')
    ax2.plot(df.index, df['Signal_Line'], label='Signal Line', color='orange')
    ax2.bar(df.index, df['MACD_Histogram'], label='MACD Histogram', color='gray', alpha=0.3)
    ax2.set_title('MACD Indicator')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

In [13]:
def plot_performance(df):
    """Plot strategy performance metrics"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Cumulative returns comparison
    strategy_cum_returns = (1 + df['Strategy_Returns']).cumprod()
    market_cum_returns = (1 + df['Returns']).cumprod()
    ax1.plot(df.index, strategy_cum_returns, label='Strategy Returns', color='blue')
    ax1.plot(df.index, market_cum_returns, label='Market Returns', color='gray', alpha=0.6)
    ax1.set_title('Cumulative Returns')
    ax1.legend()
    
    # Monthly returns heatmap
    monthly_returns = df['Strategy_Returns'].groupby([df.index.year, df.index.month]).sum().unstack()
    sns.heatmap(monthly_returns, ax=ax2, cmap='RdYlGn', center=0, annot=True, fmt='.2%')
    ax2.set_title('Monthly Returns Heatmap')
    
    # Rolling Sharpe ratio (252-day)
    rolling_sharpe = (df['Strategy_Returns'].rolling(252).mean() / 
                     df['Strategy_Returns'].rolling(252).std() * np.sqrt(252))
    ax3.plot(df.index, rolling_sharpe)
    ax3.axhline(y=0, color='r', linestyle='--')
    ax3.set_title('Rolling Sharpe Ratio (252-day)')
    
    # Drawdown analysis
    strategy_cum_returns = (1 + df['Strategy_Returns']).cumprod()
    rolling_max = strategy_cum_returns.expanding().max()
    drawdowns = (strategy_cum_returns - rolling_max) / rolling_max
    ax4.fill_between(df.index, drawdowns, 0, color='red', alpha=0.3)
    ax4.set_title('Drawdown Analysis')
    
    plt.tight_layout()
    plt.show()

In [14]:
def get_trade_info(df, strategy_name, ticker):
    """Extract trade information from the signals DataFrame"""
    trades = []
    position = 0
    entry_price = 0
    entry_date = None
    
    for date, row in df.iterrows():
        if row['Position_Change'] != 0:
            # Case 1: Opening a new position from neutral
            if position == 0:
                position = row['Position']
                entry_price = row['Close']
                entry_date = date
            # Case 2: Direct switch between long and short positions
            elif (position == 1 and row['Position'] == -1) or (position == -1 and row['Position'] == 1):
                # Close current position
                exit_price = row['Close']
                pnl = position * (exit_price - entry_price) / entry_price * 100
                trades.append({
                    'Entry Date': entry_date,
                    'Exit Date': date,
                    'Position': 'Long' if position == 1 else 'Short',
                    'Entry Price': entry_price,
                    'Exit Price': exit_price,
                    'PnL %': pnl
                })
                # Open new position
                position = row['Position']
                entry_price = row['Close']
                entry_date = date
            # Case 3: Closing a position to neutral
            elif row['Position'] == 0:
                exit_price = row['Close']
                pnl = position * (exit_price - entry_price) / entry_price * 100
                trades.append({
                    'Entry Date': entry_date,
                    'Exit Date': date,
                    'Position': 'Long' if position == 1 else 'Short',
                    'Entry Price': entry_price,
                    'Exit Price': exit_price,
                    'PnL %': pnl
                })
                position = 0
    
    trades_df = pd.DataFrame(trades)
    
    # Create ticker-specific directory
    ticker_dir = os.path.join('data', ticker)
    os.makedirs(ticker_dir, exist_ok=True)
    
    # Save trade information to CSV in ticker directory
    output_file = os.path.join(ticker_dir, f'trade_info_{strategy_name}.csv')
    trades_df.to_csv(output_file, index=False)
    print(f"\nTrade information for {strategy_name} saved to {output_file}")
    print(f"Total trades: {len(trades_df)}")
    print(f"Long trades: {len(trades_df[trades_df['Position'] == 'Long'])}")
    print(f"Short trades: {len(trades_df[trades_df['Position'] == 'Short'])}")
    
    return trades_df

In [15]:
def process_etf(etf):
    """Process a single ETF and return its results"""
    try:
        # Create ETF-specific directory
        ticker_dir = os.path.join('data', etf)
        os.makedirs(ticker_dir, exist_ok=True)
        
        # Download and process data using functions from Signals.py
        df = yf.Ticker(etf)
        df_hist = df.history(start='2005-01-01', end='2023-12-31')
        vix = yf.Ticker('^VIX')
        vix_df = vix.history(start='2005-01-01', end='2023-12-31')
        
        # Dictionary to store results for different strategies
        results = {}
        sharpe_ratios = {}
        
        # Test different MACD strategies
        strategies = {
            'MACD_Standard': lambda: get_macd_signals(df=df_hist.copy(), symbol=etf),
            'MACD_Zero_Cross': lambda: get_macd_signals_zero_cross(df=df_hist.copy(), symbol=etf),
            'VPVMA_Standard': lambda: get_vpvma_signals(df=df_hist.copy(), vix_df=vix_df.copy(), symbol=etf),
            'VPVMA_Zero_Cross': lambda: get_vpvma_signals_zero_cross(df=df_hist.copy(), vix_df=vix_df.copy(), symbol=etf)
        }
        
        for name, strategy_func in strategies.items():
            # Apply strategy
            signals_df = strategy_func()
            
            # Calculate metrics
            metrics = calculate_performance_metrics(signals_df)
            results[name] = metrics
            sharpe_ratios[name] = float(metrics['Sharpe Ratio'].replace(',', ''))
            
            # Save trade information
            get_trade_info(signals_df, name, etf)
        
        # Determine best strategy based on Sharpe ratio
        best_strategy = max(sharpe_ratios.items(), key=lambda x: x[1])[0]
        
        print(f"\nProcessed {etf}")
        return results, best_strategy, sharpe_ratios
        
    except Exception as e:
        print(f"Error processing {etf}: {str(e)}")
        return None

In [ ]:
etfs = [
    'EEM',  # Emerging Markets
    'VWO',  # Emerging Markets
    'FXI',  # China Large-Cap
    'AAXJ', # Asia ex-Japan
    'EWJ',  # Japan
    'ACWX', # All Country World ex-US
    'CHIX', # China Technology
    'CQQQ', # China Technology
    'EWZ',  # Brazil
    'ERUS', # Russia
    'EWC',  # Canada
    'EWU',  # United Kingdom
    'VGK',  # Europe
    'VPL'   # Pacific
]

# Store results for all ETFs
all_results = {}
best_strategies = {}
all_sharpe_ratios = {}

# Process ETFs sequentially
for etf in etfs:
    try:
        result = process_etf(etf)
        if result:
            results, best_strategy, sharpe_ratios = result
            best_strategies[etf] = best_strategy
            all_sharpe_ratios[etf] = sharpe_ratios
            all_results[etf] = results  # Store all results
    except Exception as e:
        print(f"Error processing {etf}: {str(e)}")

# Create summary of best strategies and key stats
summary_dir = os.path.join('data', 'summary')
os.makedirs(summary_dir, exist_ok=True)

# Save best strategies summary
with open(os.path.join(summary_dir, 'best_strategies.txt'), 'w') as f:
    f.write("Best Strategy by ETF\n")
    f.write("=" * 50 + "\n\n")
    for etf, strategy in best_strategies.items():
        f.write(f"{etf}: {strategy} (Sharpe: {all_sharpe_ratios[etf][strategy]:.2f})\n")
        f.write(f"Key Statistics:\n")
        f.write("-" * 30 + "\n")
        for metric, value in all_results[etf][strategy].items():
            f.write(f"{metric}: {value}\n")
        f.write("\n")

# Create DataFrame of all Sharpe ratios
sharpe_df = pd.DataFrame(all_sharpe_ratios).T
sharpe_df.to_csv(os.path.join(summary_dir, 'sharpe_ratios.csv'))

# Create summary statistics DataFrame
summary_stats = []
for etf in etfs:
    if etf in all_results:
        best_strat = best_strategies[etf]
        stats = all_results[etf][best_strat]
        stats['ETF'] = etf
        stats['Strategy'] = best_strat
        summary_stats.append(stats)

stats_df = pd.DataFrame(summary_stats)
stats_df.set_index('ETF', inplace=True)
stats_df.to_csv(os.path.join(summary_dir, 'strategy_statistics.csv'))

print("\nAnalysis complete. Results saved in data/summary folder.")

In [17]:

# Read the CSV file
df = pd.read_csv('data/summary/strategy_statistics.csv')

# Display all rows by setting pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Show the dataframe
df


,ETF,Number of Trades,Win Ratio,Total Return,Annual Return,Sharpe Ratio,Maximum Drawdown,Initial Portfolio Value,Final Portfolio Value,Portfolio Return,Strategy
0,EEM,164,31.71%,1046.59%,13.72%,0.70,-33.09%,"$1,000,000.00","$11,465,945.94",1046.59%,VPVMA_Zero_Cross
1,VWO,156,33.97%,973.25%,13.45%,0.67,-39.06%,"$1,000,000.00","$10,732,533.96",973.25%,VPVMA_Zero_Cross
2,FXI,174,31.61%,1579.52%,16.03%,0.68,-39.40%,"$1,000,000.00","$16,795,170.01",1579.52%,VPVMA_Zero_Cross
3,AAXJ,128,38.28%,1259.90%,18.51%,1.00,-27.81%,"$1,000,000.00","$13,598,984.51",1259.90%,VPVMA_Zero_Cross
4,EWJ,199,32.66%,163.59%,5.24%,0.38,-48.48%,"$1,000,000.00","$2,635,873.82",163.59%,VPVMA_Standard
5,ACWX,136,38.24%,408.06%,10.88%,0.68,-38.41%,"$1,000,000.00","$5,080,578.47",408.06%,VPVMA_Zero_Cross
6,CHIX,169,30.77%,329.02%,10.92%,0.55,-37.57%,"$1,000,000.00","$4,290,175.29",329.02%,VPVMA_Standard
7,CQQQ,116,25.86%,1119.08%,19.66%,0.80,-41.55%,"$1,000,000.00","$12,190,828.71",1119.08%,VPVMA_Zero_Cross
8,EWZ,213,31.92%,4960.29%,22.98%,0.78,-50.91%,"$1,000,000.00","$50,602,909.36",4960.29%,MACD_Standard
9,ERUS,104,36.54%,1069.66%,20.60%,0.72,-29.58%,"$1,000,000.00","$11,696,623.24",1069.66%,MACD_Standard
